In [1]:
import pandas as pd
import datetime
import numpy as np
import glob

# Newspaper FSI

In [2]:
FSI_file_list=glob.glob("/home/jian/Projects/Big_Lots/TMR/Circulation/Newspaper/circ_starting_2018Aug/*")

In [3]:
newspaper=pd.DataFrame()
for file in FSI_file_list:
    month=file.split("/")[9].split(" ")[0]
    day=file.split("/")[9].split(" ")[1]
    year=file.split("/")[9].split(" ")[2]
    if month=="Feb":
        month="February"
    if month=="Nov":
        month="November"
    date_str=month+" "+day+" "+year
    try:
        date_in_file=datetime.datetime.strptime(date_str,"%B %d %Y").date()
    except:
        print("file_name_error: "+date_str)
    if date_in_file>=datetime.date(2016,10,2):
        df=pd.read_excel(file,dtype=str)
        df.columns=[x.upper() for x in df.columns.tolist()]
        
        try:
            df=df[['DMA','CONFIRMED CIRCULATION','MEDIA COST','PRINT COST']]
        except:
            print("columns_name_error: "+date_str)
        df=df[df['DMA']!='nan']
        
        df['CONFIRMED CIRCULATION']=df['CONFIRMED CIRCULATION'].astype(float)
        df['MEDIA COST']=df['MEDIA COST'].astype(float)
        df['PRINT COST']=df['PRINT COST'].astype(float)
        df['cost']=df['MEDIA COST']+df['PRINT COST']
        del df['MEDIA COST']
        del df['PRINT COST']
        
        df=df.groupby(['DMA'])['CONFIRMED CIRCULATION','cost'].sum().reset_index()
        
        
        if date_in_file.weekday()==6:
            week_start_date=date_in_file
            week_end_saturday=date_in_file+datetime.timedelta(days=6)
        else:
            week_start_date=date_in_file-datetime.timedelta(days= date_in_file.weekday()+1)
            week_end_saturday=week_start_date+datetime.timedelta(days=6)
        df['week_start_date']=week_start_date
        df['week_end_date']=week_end_saturday
        df['file_date']=date_in_file
        newspaper=newspaper.append(df)
    
    

In [4]:
newspaper_DMA=newspaper.copy()
del newspaper_DMA['file_date']
newspaper_DMA['media']='Circulation'
newspaper_DMA['submedia']='FSI+Instore'
newspaper_DMA['placement']="xx"
newspaper_DMA=newspaper_DMA.rename(columns={"CONFIRMED CIRCULATION":"impression"})
newspaper_DMA=newspaper_DMA[['week_start_date','week_end_date','DMA','media','submedia','placement','impression','cost']]


newspaper_national=newspaper.groupby(['week_start_date','week_end_date'])['CONFIRMED CIRCULATION','cost'].sum().reset_index()
newspaper_national=newspaper_national.rename(columns={"CONFIRMED CIRCULATION":"impression"})
newspaper_national['media']='Circulation'
newspaper_national['submedia']='FSI+Instore'
newspaper_national['placement']="xx"
newspaper_national=newspaper_national[['week_start_date','week_end_date','media','submedia','placement','impression','cost']]


# Valassis

In [5]:
Valassis_file_list=glob.glob("/home/jian/Projects/Big_Lots/TMR/Circulation/Valassis/valassis_starting_2018Aug/*")

In [6]:
valassis=pd.DataFrame()
for file in Valassis_file_list:
    file_name=file.split("/")[9]
    if file_name[3]=="_":
        file_date_name=file_name[9:]
        month=file_date_name.split(" ")[0]
        day=file_date_name.split(" ")[1]
        year=file_date_name.split(" ")[2]
        if len(year)==2:
            year="20"+year
        if ((len(month)==2) & (month[0]=='0')):
            month=month[1]
        if ((len(day)==2) & (day[0]=='0')):
            day=day[1]
        date_in_file=datetime.date(int(year),int(month),int(day))
    elif file_name[3]==" ":
        file_date_name=file_name[9:].split(" ")[0]
        month=file_date_name.split(".")[0]
        day=file_date_name.split(".")[1]
        year=file_date_name.split(".")[2]
        if len(year)==2:
            year="20"+year
        if ((len(month)==2) & (month[0]=='0')):
            month=month[1]
        if ((len(day)==2) & (day[0]=='0')):
            day=day[1]
        date_in_file=datetime.date(int(year),int(month),int(day))
    else:
        print("file date detect failed: "+str(file))
    if date_in_file>=datetime.date(2016,10,2):
        df=pd.read_excel(file,dtype=str)
        start_row=df[df.iloc[:,0].apply(lambda x: x.lower())=="name"].reset_index()['index'].tolist()[0]
        df=pd.read_excel(file,dtype=str,skiprows=(start_row+1))
        df.columns=[x.lower() for x in df.columns.tolist()]
        df=df[df['description']!='nan']
        if "shared count" in df.columns.tolist():
            df=df[['description','shared count']].rename(columns={"description":"DMA","shared count":"impression"})
            df['date_in_file']=date_in_file
            df['impression']=df['impression'].astype(float)
            df=df.groupby(['date_in_file','DMA'])['impression'].sum().to_frame().reset_index()
            
            if date_in_file.weekday()==6:
                week_start_date=date_in_file
                week_end_saturday=date_in_file+datetime.timedelta(days=6)
            else:
                week_start_date=date_in_file-datetime.timedelta(days= date_in_file.weekday()+1)
                week_end_saturday=week_start_date+datetime.timedelta(days=6)
            df['week_start_date']=week_start_date
            df['week_end_date']=week_end_saturday
            valassis=valassis.append(df)
            
        elif "shared counts" in df.columns.tolist():
            df=df[['description','shared counts']].rename(columns={"description":"DMA","shared counts":"impression"})
            df['date_in_file']=date_in_file
            df['impression']=df['impression'].astype(float)
            df=df.groupby(['date_in_file','DMA'])['impression'].sum().to_frame().reset_index()

            if date_in_file.weekday()==6:
                week_start_date=date_in_file
                week_end_saturday=date_in_file+datetime.timedelta(days=6)
            else:
                week_start_date=date_in_file-datetime.timedelta(days= date_in_file.weekday()+1)
                week_end_saturday=week_start_date+datetime.timedelta(days=6)
            df['week_start_date']=week_start_date
            df['week_end_date']=week_end_saturday
            valassis=valassis.append(df)
        else:
            print(file_name)

In [7]:
valassis['cost']=0
valassis_DMA=valassis.copy()
del valassis_DMA['date_in_file']
valassis_DMA['media']='Circulation'
valassis_DMA['submedia']='Valassis'
valassis_DMA['placement']="xx"

valassis_DMA=valassis_DMA[['week_start_date','week_end_date','DMA','media','submedia','placement','impression','cost']]


valassis_national=valassis.groupby(['week_start_date','week_end_date'])['impression','cost'].sum().reset_index()
valassis_national['media']='Circulation'
valassis_national['submedia']='Valassis'
valassis_national['placement']="xx"
valassis_national=valassis_national[['week_start_date','week_end_date','media','submedia','placement','impression','cost']]

# Output

In [8]:
circulation_TMR_new=newspaper_DMA.append(valassis_DMA)
circulation_TMR_old=pd.read_csv("/home/jian/Projects/Big_Lots/TMR/Circulation/output/circulation_TMR_JL_2018-08-14.csv",dtype=str)
circulation_TMR_old['week_start_date']=circulation_TMR_old['week_start_date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
circulation_TMR_old['week_end_date']=circulation_TMR_old['week_end_date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
circulation_TMR_old['impression']=circulation_TMR_old['impression'].astype(float)
circulation_TMR_old['cost']=circulation_TMR_old['cost'].astype(float)
circulation_TMR_all=circulation_TMR_old.append(circulation_TMR_new)

In [9]:
circulation_TMR_all.to_csv("/home/jian/Projects/Big_Lots/TMR/Circulation/output/circulation_TMR_JL_"+str(datetime.datetime.now().date())+".csv",index=False)

national_up_to_Q4_2018=circulation_TMR_all.groupby(['week_start_date','week_end_date'])['impression','cost'].sum().reset_index()
national_up_to_Q4_2018.to_csv("/home/jian/Projects/Big_Lots/TMR/Circulation/output/circulation_natioanl_agged_JL_"+str(datetime.datetime.now().date())+".csv",index=False)
